In [2]:
import albumentations as alb
import os
import time
import cv2
import json  
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model

In [4]:
facetracker = load_model('facetracker.h5')

TypeError: ('Keyword argument not understood:', 'keepdims')